#Dataset Preprocessing


*   Since our previous data exploration showed the need for more predictors, we will be merging more datasets
*   Handling missing values
* Ultimately creating one cohesive + cleaned dataset that we can use for pipelines + ML models



In [2]:
import pandas as pd
import numpy as np

# load the data

from google.colab import drive
drive.mount('/content/drive')
merged_OP = pd.read_csv("/content/drive/MyDrive/Snowy Plover Datasets/merged_OP.csv")

# More datasets
df_events  = pd.read_csv("/content/drive/MyDrive/Snowy Plover Datasets/SFAN_SNPL_Events.csv")
df_nesting = pd.read_csv("/content/drive/MyDrive/Snowy Plover Datasets/SFAN_SNPL_Nesting.csv")
df_chicks  = pd.read_csv("/content/drive/MyDrive/Snowy Plover Datasets/SFAN_SNPL_ChickBands.csv")
df_bands   = pd.read_csv("/content/drive/MyDrive/Snowy Plover Datasets/SFAN_SNPL_Bands.csv")

print("merged_OP:", merged_OP.shape)
print("events:   ", df_events.shape)
print("nesting:  ", df_nesting.shape)
print("chicks:   ", df_chicks.shape)



Mounted at /content/drive
merged_OP: (13220, 51)
events:    (3339, 37)
nesting:   (846, 68)
chicks:    (370, 20)


In [3]:
# Verify that start_date is datetime in the previous merged dataset
if "Start_Date" in merged_OP.columns:
    merged_OP["Start_Date"] = pd.to_datetime(merged_OP["Start_Date"], errors = "coerce")
    merged_OP["Year"] = merged_OP["Start_Date"].dt.year
    merged_OP["Month"] = merged_OP["Start_Date"].dt.month
    merged_OP["DayOfYear"] = merged_OP["Start_Date"].dt.dayofyear

# Events dates
if "Start_Date" in df_events.columns:
    df_events["Start_Date"] = pd.to_datetime(df_events["Start_Date"], errors="coerce")
if "Start_Year" in df_events.columns:
    df_events["Year"] = df_events["Start_Year"]

# Nesting important dates
for col in ["Date_Found", "Hatching_Date", "Fledging_Date", "Failure_Date"]:
    if col in df_nesting.columns:
        df_nesting[col] = pd.to_datetime(df_nesting[col], errors="coerce")



In [5]:
#merge events dataset

event_cols = ["Event_ID", "Survey_Direction", "Wind_Spd_mph","Wind_Max_mph", "Wind_Dir", "Air_Temp_degF",
    "Rel_Hum_per",
    "Cloud_Cover",
    "Tide_Cond",
    "Incomplete_Survey"
]

df_events_small = df_events.loc[:, [c for c in event_cols if c in df_events.columns]]
df_events_small = df_events_small.drop_duplicates(subset = ["Event_ID"])

merged_env = merged_OP.merge(df_events_small, on = "Event_ID", how = "left")
print("Shape after events merge:", merged_env.shape)


Shape after merge: (13220, 61)


In [6]:
#merge nesting dataset

if "Nest_ID" in merged_env.columns:
    nest_cols = ["Nest_ID",  "Eggs", "Hatchlings",  "Fledglings",  "Failure_Reason", "Predator_Type", "Restored_Area"]
    nest_cols = [c for c in nest_cols if c in df_nesting.columns]

    df_nest_small = df_nesting[nest_cols].drop_duplicates(subset = ["Nest_ID"])

    # Check is fledglings is working
    if "Fledglings" in df_nest_small.columns:
        df_nest_small["NestSuccess"] = (df_nest_small["Fledglings"].fillna(0) > 0).astype(int)

    merged_env_nest = merged_env.merge(df_nest_small, on = "Nest_ID", how = "left")
else:
    merged_env_nest = merged_env.copy()

print("Shape after nesting merge:", merged_env_nest.shape)


Shape after nesting merge: (13220, 68)


In [7]:
#merge chickbands aggregates

if "Nest_ID" in df_chicks.columns and "Nest_ID" in merged_env_nest.columns:
    df_chicks["EggToothPresentFlag"] = df_chicks["EggToothPresence"].astype(str)\
        .str.contains("Yes", case = False, na = False).astype(int)

    chick_agg = (
        df_chicks.groupby("Nest_ID")
        .agg(
            NumChicks_Banded    = ("Chick_BandData_IDInt", "count"),
            MeanChickWeight_g   = ("ChickWeight_g", "mean"),
            Pct_Chicks_EggTooth = ("EggToothPresentFlag", "mean"),
        )
        .reset_index()
    )

    merged_full = merged_env_nest.merge(chick_agg, on = "Nest_ID", how = "left")
else:
    merged_full = merged_env_nest.copy()

print("Shape after Chickband merge:", merged_full.shape)


Shape after Chickband merge: (13220, 71)


In [8]:
#Generic features usable for future models

# Total plovers in that event
snpl_cols = ["SNPL_Male", "SNPL_Female", "SNPL_Unk", "SNPL_Hatchlings", "SNPL_Fledglings"]
for c in snpl_cols:
    if c not in merged_full.columns:
        merged_full[c] = 0

merged_full["Total_SNPL"] = merged_full[snpl_cols].sum(axis=1)

#Verifying predator present
if "Predator_Present" in merged_full.columns:
    merged_full["Predator_Present"] = merged_full["Predator_Present"].fillna(False)


In [9]:
#Missing values / missing inputs

df = merged_full

# Fill the count like columns
count_like_cols = ["SNPL_Male", "SNPL_Female", "SNPL_Unk", "SNPL_Hatchlings", "SNPL_Fledglings", "Total_SNPL","NumPredator", "Eggs", "Hatchlings", "Fledglings",
    "NumChicks_Banded"]

for c in count_like_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

# Fill with median in continuous columns
continuous_cols = [ "Air_Temp_degF", "Rel_Hum_per", "Wind_Spd_mph", "Wind_Max_mph", "MeanChickWeight_g", "Pct_Chicks_EggTooth"]

for c in continuous_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
        median_val = df[c].median()
        df[c] = df[c].fillna(median_val)

# Deal with booleans
bool_like_cols = ["Predator_Present", "Incomplete_Survey", "NestSuccess", "Restored_Area"]
for c in bool_like_cols:
    if c in df.columns:
        df[c] = df[c].map({True: True, False: False, "Yes": True, "No": False}).fillna(False)
        df[c] = df[c].astype(bool)

# Fill with unknown
cat_cols = [ "Loc_Name", "Loc_Code","Tide_Cond", "Wind_Dir", "Survey_Direction","Failure_Reason", "Predator_Type", "Observers"
]

for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip()
        df[c] = df[c].replace({"nan": np.nan, "None": np.nan})
        df[c] = df[c].fillna("Unknown")
        df[c] = df[c].astype("category")


/tmp/ipython-input-976887777.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].map({True: True, False: False, "Yes": True, "No": False}).fillna(False)
/tmp/ipython-input-976887777.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].map({True: True, False: False, "Yes": True, "No": False}).fillna(False)
/tmp/ipython-input-976887777.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the

In [10]:
# Saving cleaned dataset

cleaned_df = df.copy()

cleaned_path = "/content/drive/MyDrive/Snowy Plover Datasets/cleaned_plover_dataset.csv"
cleaned_df.to_csv(cleaned_path, index = False)

print("Saved cleaned dataset to:", cleaned_path)
print("Final shape:", cleaned_df.shape)
print("Columns:", cleaned_df.columns.tolist())


Saved cleaned dataset to: /content/drive/MyDrive/Snowy Plover Datasets/cleaned_plover_dataset.csv
Final shape: (13220, 71)
Columns: ['ID', 'SNPL_Data_ID', 'Event_ID', 'Type', 'Basis_Of_Record', 'Unit_Code', 'Location_ID', 'Loc_Code', 'Loc_Name', 'Start_Year', 'Start_Date', 'Nest_ID', 'SNPL_Time', 'SNPL_Time_Local', 'Local_Time_Zone', 'QCFlag', 'QCNotes', 'Waypoint', 'Decimal_Latitude', 'Decimal_Longitude', 'Coord_System', 'Datum', 'Scientific_Name', 'SNPL_Male', 'SNPL_Female', 'SNPL_Unk', 'SNPL_Hatchlings', 'SNPL_Fledglings', 'SNPL_Bands', 'Number_Eggs', 'BehaviorTerritoryCC', 'BehaviorTerritoryLW', 'BehaviorTerritoryMD', 'BehaviorTerritorySC', 'BehaviorTerritoryCP', 'BehaviorNestCP', 'BehaviorNestDC', 'BehaviorNestAI', 'BehaviorNestFN', 'BehaviorChicksAC', 'BehaviorChicksNA', 'BehaviorOtherFG', 'SNPL_Notes', 'Year', 'PredatorType', 'CommonName', 'NumPredator', 'PredatorAction', 'Predator_Present', 'Total_SNPL', 'Month', 'DayOfYear', 'Survey_Direction', 'Wind_Spd_mph', 'Wind_Max_mph', 